In [2]:
from pyspark import SparkContext

sc=SparkContext(master="local",appName="Test")


In [3]:
uindf=sc.textFile("UINCardData.csv")
uindf.take(6)

['UIN00001,982120000,Male,N,65000',
 'UIN00002,982120001,Male,N,35000',
 'UIN00003,982120002,Male,N,34000',
 'UIN00004,982120003,Male,Y,44000',
 'UIN00005,982120004,Male,N,54000',
 'UIN00006,982120005,Male,N,5000']

In [5]:
bankdf=sc.textFile("BankAccountLink.csv")
bankdf.take(6)

['982120000,Y,20004562111',
 '982120001,Y,20004562112',
 '982120002,Y,20004562113',
 '982120003,Y,20004562114',
 '982120004,Y,20004562115',
 '982120005,Y,20004562116']

In [11]:


# Read files
uindf = sc.textFile("UINCardData.csv")
bankdf = sc.textFile("BankAccountLink.csv")

# Remove header
uin_header = uindf.first()
bank_header = bankdf.first()

uindf = uindf.filter(lambda x: x != uin_header)
bankdf = bankdf.filter(lambda x: x != bank_header)

# Convert to pair RDD using MobileNumber as key
uin_rdd = uindf.map(lambda x: x.split(",")).map(lambda x: (x[1], x[0]))
# (MobileNumber, UIN)

bank_rdd = bankdf.map(lambda x: x.split(",")).map(lambda x: (x[0], x[2]))
# (MobileNumber, BankAccountNumber)

# Left Outer Join
joined = uin_rdd.leftOuterJoin(bank_rdd)

# Filter where BankAccountNumber is None
not_linked = joined.filter(lambda x: x[1][1] is None)

# Extract required columns
final = not_linked.map(lambda x: (x[1][0], x[1][1]))

# Print result
print(final.collect())


[]
